In [1]:
import datetime
import pandas as pd

df = pd.read_csv("Bainbridge_Departures_2013_with_Seconds_Late.csv", parse_dates = ["timestamps"], index_col = "timestamps")

In [2]:
df.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime,seconds_late
timestamps,,,,,,,
2013-01-01 00:55:00,Bainbridge,Colman,55:00.0,05:56.0,37:37.0,32.0,656.0
2013-01-01 04:45:00,Bainbridge,NaN,45:00.0,NaN,NaN,NaN,NaN
2013-01-01 05:20:00,Bainbridge,Colman,20:00.0,19:01.0,49:40.0,30.0,-59.0
2013-01-01 06:20:00,Bainbridge,NaN,20:00.0,NaN,NaN,NaN,NaN
2013-01-01 07:05:00,Bainbridge,NaN,05:00.0,06:16.0,NaN,NaN,76.0


This time series data is not consistent. The goal of the notebook is to use Linear Interpolation to generate consiste timestamps for every 5 minutes.

First I will generate more timestamps for evey 5 minute interval.

<a href = "https://stackoverflow.com/questions/13295735/how-can-i-replace-all-the-nan-values-with-zeros-in-a-column-of-a-pandas-datafra">How can I replace all the NaN values with Zero's in a column of a pandas dataframe</a>

In [3]:
def generate_filler_timestamps(df, step = 5):
    prev_date = df.iloc[0].name

    tdelta = datetime.timedelta(minutes = step)

    original_indicies = df.index.to_series()

    new_timestamps = []

    while (not prev_date == datetime.datetime(2013, 12, 31, 23, 55)):
        if(prev_date in original_indicies):
            prev_date = prev_date + tdelta
            continue
        new_timestamps.append(prev_date)
        prev_date = prev_date + tdelta

    new_timestamps.append(datetime.datetime(2013, 12, 31, 23,55,0))

    df2 = pd.DataFrame(index = new_timestamps, columns=["departure_terminal", "arrival_terminal",
                                                        "scheduled_departure", "actual_departure",
                                                        "Actual_Arrival", "TransitTime", "seconds_late"])
    df2["departure_terminal"].fillna("Bainbridge", inplace = True)
    df2["arrival_terminal"].fillna("Colman", inplace = True)
    df2.index.rename("timestamps", inplace = True)

    df3 = df.append(df2)
    df3 = df3.sort_values(by = "timestamps")
    
    return df3

In [4]:
dfLarge = generate_filler_timestamps(df)

In [5]:
dfLarge.head()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime,seconds_late
timestamps,,,,,,,
2013-01-01 00:55:00,Bainbridge,Colman,55:00.0,05:56.0,37:37.0,32.0,656.0
2013-01-01 01:00:00,Bainbridge,Colman,NaN,NaN,NaN,NaN,NaN
2013-01-01 01:05:00,Bainbridge,Colman,NaN,NaN,NaN,NaN,NaN
2013-01-01 01:10:00,Bainbridge,Colman,NaN,NaN,NaN,NaN,NaN
2013-01-01 01:15:00,Bainbridge,Colman,NaN,NaN,NaN,NaN,NaN


In [6]:
dfLarge.tail()

,departure_terminal,arrival_terminal,scheduled_departure,actual_departure,Actual_Arrival,TransitTime,seconds_late
timestamps,,,,,,,
2013-12-31 23:35:00,Bainbridge,Colman,NaN,NaN,NaN,NaN,NaN
2013-12-31 23:40:00,Bainbridge,Colman,NaN,NaN,NaN,NaN,NaN
2013-12-31 23:45:00,Bainbridge,Colman,NaN,NaN,NaN,NaN,NaN
2013-12-31 23:50:00,Bainbridge,Colman,NaN,NaN,NaN,NaN,NaN
2013-12-31 23:55:00,Bainbridge,Colman,NaN,NaN,NaN,NaN,NaN


Now I want to use Linear Interpolation to fill in some of these `NaN` values

In [15]:
dfLarge["seconds_late"] = dfLarge["seconds_late"].interpolate()

In [20]:
dfLarge.to_csv("Bainbridge_Departures_2013_Interpolated.csv")